### Previsão de tarifa de táxi de Nova Iorque

In [ ]:
# Instalação das bibliotecas que vamos utilizar
#!apt-get install libgeos-3.6.2
#!apt-get install libgeos-dev
#!pip install https://github.com/matplotlib/basemap/archive/master.zip


In [ ]:
import pandas as pd
import numpy as np
import random
import zipfile
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
numero_de_linhas_no_arquivo= sum(1 for line in open('../input/new-york-city-taxi-fare-prediction/train.csv'))
numero_de_linhas_no_arquivo

In [ ]:
reduzir_amostra = 250000

In [ ]:
#Reduzindo a base
range(1, numero_de_linhas_no_arquivo + 1)
numero_de_linhas_no_arquivo - reduzir_amostra

In [ ]:
#randomizando a base e pegando os elementos
random.seed(24)
nova_base = sorted(random.sample(range(1, numero_de_linhas_no_arquivo + 1),
                                 numero_de_linhas_no_arquivo - reduzir_amostra))

In [ ]:
numero_de_linhas_no_arquivo - len(nova_base)

In [ ]:
#Nova base
train = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv', skiprows = nova_base)
train.shape

In [ ]:
# Salvando a nova base de dados 
train.to_csv('train_sample_' + str(reduzir_amostra) + '.csv', index = False)

In [ ]:
train = pd.read_csv('./train_sample_250000.csv',
                    parse_dates = ['key', 'pickup_datetime'],
                    dtype = {'fare_amount': np.float32,
                             'pickup_latitude': np.float32,
                             'pickup_longitude': np.float32,
                             'dropoff_latitude': np.float32,
                             'dropoff_longitude': np.float32,
                             'passenger_count': np.int32             
                    })

In [ ]:
train.head()

In [ ]:
train.dtypes

## Limpando a base

In [ ]:
# verificando se tem valores null
train.isna().sum(axis = 0)

In [ ]:
key = train['key']
key.describe()

In [ ]:
#Analisando as datas
plt.hist(key, bins = 100)

In [ ]:
pdt = train['pickup_datetime']
pdt.describe()

In [ ]:
#Analisando as datas
plt.hist(pdt, bins = 100)

In [ ]:
pdt = pdt.map(lambda date: date.tz_localize(None))

In [ ]:
pdt.head()

In [ ]:
key = pd.to_datetime(key.dt.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
pdt = pd.to_datetime(pdt.dt.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
key.head()

In [ ]:
#Notamos que a coluna é a msm podendo ser dropada da base de dados
key == pdt
    

In [ ]:
#dropando key
train.drop(['key'], axis = 1, inplace = True)

In [ ]:
train.head()

In [ ]:
# Padronizando os dados
train['pickup_datetime'] = pdt
train.head()

In [ ]:
#Cordenadas longitude da origem 
plongitude = train['pickup_longitude']
plongitude.describe()

In [ ]:
plt.hist(plongitude, bins = 100)

In [ ]:
#Valores inconsistentes
plongitude.median()

In [ ]:
plt.hist(plongitude[(plongitude > plongitude.median() -2.5) & (plongitude < plongitude.median() +2.5)], bins = 100)

In [ ]:
plon_var = ((plongitude > plongitude.median() -0.5) & (plongitude < plongitude.median() +0.5))

In [ ]:
plon_var

In [ ]:
plon_var.value_counts()

In [ ]:
print(plon_var.value_counts(), '\n', plon_var.value_counts(normalize= True))

In [ ]:
#origem latitute
platitude = train['pickup_latitude']
platitude.describe()

In [ ]:
plt.hist(platitude, bins = 100)

In [ ]:
plt.hist(platitude[(platitude > platitude.median() -2.5) & (platitude < platitude.median() +2.5)], bins = 100)

In [ ]:
platitude_var = ((platitude > platitude.median() -0.5) & (platitude < platitude.median() +0.5))
print(platitude_var.value_counts(), '\n', platitude_var.value_counts(normalize= True))

In [ ]:
#chegada latitude
platitude_chegada = train['dropoff_latitude']
platitude_chegada.describe()
plt.hist(platitude_chegada, bins = 100)

In [ ]:
platitude_chegada.describe()

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,5))
ax[0].hist(platitude_chegada, bins = 100)
ax[1].hist(platitude_chegada[(platitude_chegada > platitude_chegada.median() -2.5) & (platitude_chegada < platitude_chegada.median() +2.5)], bins = 100)

In [ ]:
platitude_chegada_var = ((platitude_chegada > platitude_chegada.median() -0.5) & (platitude_chegada < platitude_chegada.median() +0.5))
print(platitude_chegada_var.value_counts(), '\n', platitude_chegada_var.value_counts(normalize= True))

In [ ]:
# chegada longitude
plongitude_chegada = train['dropoff_longitude']
plt.hist(plongitude_chegada, bins = 100)

In [ ]:
plongitude_chegada.describe()

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,5))
ax[0].hist(plongitude_chegada, bins = 100)
ax[1].hist(plongitude_chegada[(plongitude_chegada > plongitude_chegada.median() -2.5) & (plongitude_chegada < plongitude_chegada.median() +2.5)], bins = 100)

In [ ]:
plongitude_chegada_var = ((plongitude_chegada > plongitude_chegada.median() -0.5) & (plongitude_chegada < plongitude_chegada.median() +0.5))
print(plongitude_chegada_var.value_counts(), '\n', plongitude_chegada_var.value_counts(normalize= True))

In [ ]:
# contador
count_analise = train['passenger_count']
count_analise.describe()

In [ ]:
# deu algo errado nos dados
print((count_analise ==0).sum(), (count_analise == 0).mean())

In [ ]:
plt.hist(count_analise, bins = 100)

In [ ]:
plt.hist(count_analise[count_analise < 20], bins = 100)

In [ ]:
count_analise_val = (count_analise>= 1) & (count_analise <=6)
print(count_analise_val.value_counts(), '\n', count_analise_val.value_counts(normalize = True))

In [ ]:
# preço da viagem analise
preco_viagem = train['fare_amount']
preco_viagem.describe()

In [ ]:
plt.hist(preco_viagem, bins = 100)

In [ ]:
plt.hist(preco_viagem[preco_viagem < 10], bins = 100)

In [ ]:
plt.hist(preco_viagem[preco_viagem > 50], bins = 100)

In [ ]:
preco_viagem_val = (preco_viagem>= 1) & (preco_viagem <=400)
print(preco_viagem_val.value_counts(), '\n', preco_viagem_val.value_counts(normalize = True))

In [ ]:
#Nova base com as alteracoes
nova_base = plon_var & platitude_var& platitude_chegada_var& plongitude_chegada_var& count_analise_val&preco_viagem_val

In [ ]:
print(nova_base.value_counts(),'\n',nova_base.value_counts(normalize = True) )

In [ ]:
train = train.drop(nova_base[nova_base == False].index)

In [ ]:
train.head()

In [ ]:
train.shape

### Se analisamos em uma corrida de taxi, temos dois fatores que são a distância e o tempo da corrida para definir o preço.
1. Não é apresentado o dado do tempo de corrida
2. Como o intuito é prever quando pedir a corrida, não tem sentido saber o tempo exato pq pode mudar a cada horário
3. Mas temos algumas informações importantes pickup_datetime que é horário, com isso podemos deduzir;
    * hora do dia que tem maior tráfego,  dia da semana de maior movimento, feriados, ano devida a economia da região etc

In [ ]:
train['hora_dia'] = train['pickup_datetime'].map(lambda date: date.timetuple().tm_hour)

In [ ]:
train

In [ ]:
train['dia_semana'] = train['pickup_datetime'].map(lambda date: date.timetuple().tm_wday)
train['dias_do_ano'] = train['pickup_datetime'].map(lambda date: date.timetuple().tm_yday)
train['ano'] = train['pickup_datetime'].map(lambda date: date.timetuple().tm_year)
train['mes'] = train['pickup_datetime'].map(lambda date: date.timetuple().tm_mon)

In [ ]:
train

In [ ]:
#remover a zorra do date e foda se
len(train['pickup_datetime'][train['pickup_datetime'].dt.strftime('%m-%d') == '02-29'])

In [ ]:
train.drop(train['pickup_datetime'][train['pickup_datetime'].dt.strftime('%m-%d') == '02-29'].index,
           inplace = True)

In [ ]:
train.shape

In [ ]:
fig, ax = plt.subplots(1, 5, figsize = (15,3))
train_not_2015 = train[train['ano'] < 2015]
ax[0].hist(train_not_2015['hora_dia'], bins = 24)
ax[0].set_title('Hora do dia')
ax[1].hist(train_not_2015['dia_semana'], bins = 7)
ax[1].set_title('Dia da semana')
ax[2].hist(train_not_2015['dias_do_ano'], bins = 365)
ax[2].set_title('Quantidade de dias do ano')
ax[3].hist(train_not_2015['ano'], bins = 6)
ax[3].set_title('Ano')
ax[4].hist(train_not_2015['mes'], bins = 12)
ax[4].set_title('mes')

In [ ]:
plt.figure(figsize = (15, 5))
plt.scatter(train['pickup_datetime'], train['fare_amount'], s = 1, alpha = 0.2)

In [ ]:
from collections import Counter
tarifa_media_amostra = train['fare_amount'][(train['fare_amount'] > 40) & (train['fare_amount'] < 60)]

In [ ]:
tarifa_media_amostra_count = Counter(tarifa_media_amostra)
tarifa_media_amostra_count

In [ ]:
tarifa_media_amostra_mais_comum = tarifa_media_amostra_count.most_common(10)
tarifa_media_amostra_mais_comum

In [ ]:
plt.bar([x[0] for x in tarifa_media_amostra_mais_comum], [x[1] for x in tarifa_media_amostra_mais_comum])

In [ ]:
#!pip install pygal
#!pip install cairosvg
#!pip install geopandas
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev -y
!pip install https://github.com/matplotlib/basemap/archive/master.zip
#!pip install pyproj==1.9.6

In [ ]:
import mpl_toolkits.basemap as mp

In [ ]:
mp.__version__

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
fig = plt.figure(figsize=(15, 8))
m = Basemap(projection='lcc', resolution='c',
            lat_0=-17, lon_0=-47.92,
            width=8000000, height=5000000)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
x, y = m(-47.92, -15.78)
plt.plot(x, y, 'ok', markersize=5)

In [ ]:
#conda install -c conda-forge basemap-data-hires

In [ ]:
#NYC
lat1, lat2 = 40.55, 40.95
lon1, lon2 = -74.10, -73.70

fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='cyl', resolution='c', llcrnrlon = lon1, llcrnrlat = lat1, urcrnrlon= lon2, urcrnrlat = lat2)
m.shadedrelief()
m.fillcontinents(color = 'palegoldenrod', lake_color = 'lightskyblue')
m.drawmapboundary(fill_color = 'lightskyblue')
m.drawparallels(np.arange(lat1, lat2 + 0.05, 0.1), labels = [1, 0, 0, 0])
m.drawmeridians(np.arange(lon1, lon2 + 0.05, 0.1), labels = [0, 0, 0, 1])

# Pickup locations - de todas as saídas (verde)
m.scatter(train['pickup_longitude'], train['pickup_latitude'], s = 1, c = 'green',
          alpha = 0.1, zorder = 5)
# Dropoof locations - de todas as saídas (amarelo)
m.scatter(train['dropoff_longitude'], train['dropoff_latitude'], s = 1, c='yellow',
         alpha = 0.1, zorder = 5)
for i in [0, 1, 2, 4]:
  this_fare = tarifa_media_amostra_mais_comum[i][0]
  this_df = train[train['fare_amount'] == this_fare]
  # pickup location - vermelha
  m.scatter(this_df['pickup_longitude'], this_df['pickup_latitude'], s = 2, c = 'red',
           alpha = 0.2, zorder = 5)
  # dropoff location - azul
  m.scatter(this_df['dropoff_longitude'], this_df['dropoff_latitude'], s = 2, c = 'blue',
           alpha = 0.2, zorder = 5)
  

In [ ]:
coords = train[['dropoff_latitude',
                'dropoff_longitude']][(train['fare_amount'] > 40) &
                                       (train['fare_amount'] < 60) &
                                       (train['dropoff_latitude'] < 40.7) &
                                       (train['dropoff_latitude'] > 40.6) &
                                       (train['dropoff_longitude'] < -73.7) &
                                       (train['dropoff_longitude'] > -73.9)]

In [ ]:
coords.shape

In [ ]:
coords.head

In [ ]:
print(coords['dropoff_latitude'].median(), coords['dropoff_longitude'].median())

In [ ]:
filtered = train[['fare_amount', 
                  'passenger_count']][((train['fare_amount'] == tarifa_media_amostra_mais_comum[0][0]) |
                                       (train['fare_amount'] == tarifa_media_amostra_mais_comum[1][0]) |
                                       (train['fare_amount'] == tarifa_media_amostra_mais_comum[2][0]) |
                                       (train['fare_amount'] == tarifa_media_amostra_mais_comum[4][0]))&
                                       (train['fare_amount'] < 60) &
                                       (train['dropoff_latitude'] < 40.7) & 
                                       (train['dropoff_latitude'] > 40.6) & 
                                       (train['dropoff_longitude'] < -73.7) &
                                       (train['dropoff_longitude'] > -73.9)]

In [ ]:
train.head()

In [ ]:
train.drop('pickup_datetime', axis = 1, inplace = True)

In [ ]:
train.head()

In [ ]:
lon1_origem, lon2_origem = np.radians(train['pickup_longitude']), np.radians(train['dropoff_longitude'])

In [ ]:
lat1_destino, lat2_destino = np.radians(train['pickup_latitude']), np.radians(train['dropoff_latitude'])

In [ ]:
dlon = lon2_origem - lon1_origem
dlat = lat2_destino - lat1_destino

In [ ]:
# Distância Euclidiana (Km)
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
train['distancia_euclidi_da_massa'] = 6373 * c

In [ ]:
train.head()

In [ ]:
X_train = train.drop('fare_amount', axis = 1)

In [ ]:
y_train = train['fare_amount']

In [ ]:
y_train.head()

In [ ]:
X_train.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_escala_normaliza = scaler.transform(X_train)

In [ ]:
X_train_escala_normaliza

In [ ]:
# Validação cruzada, usei 3 folds, fazendo três testes e tendo tres resultado e fazendo a media
from sklearn.model_selection import cross_val_score
cv = 3
scoring = 'neg_mean_squared_error'
import multiprocessing
n_jobs = multiprocessing.cpu_count() - 1

In [ ]:
#Linear Vai
from sklearn.linear_model import LinearRegression
model = LinearRegression()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
print('Média dos tres testes: ', scores.mean())

In [ ]:
# Ridge
from sklearn.linear_model import Ridge
model = Ridge()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
print('Média dos tres testes: ', scores.mean())

In [ ]:
#Lasso
from sklearn.linear_model import Lasso
model = Lasso()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
print('Média dos tres testes: ', scores.mean())

In [ ]:
#baseado em instancias
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
print('Média dos tres testes: ', scores.mean())

In [ ]:
#arvore de decisao
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
#Floresta randomicas
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
print('Média dos tres testes: ', scores.mean())

In [ ]:
#GradientBoost
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
print('Média dos tres testes: ', scores.mean())

In [ ]:
#Deep
from sklearn.neural_network import MLPRegressor
model = MLPRegressor()
scores = cross_val_score(model, X_train_escala_normaliza, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
np.sqrt(-scores.mean())

In [ ]:
print('Média dos tres testes: ', scores.mean())

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test, y_train1, y_test = train_test_split(X_train_escala_normaliza, y_train,
                                                      test_size = 0.2, random_state = 24)

In [ ]:
X_train1.shape

In [ ]:
X_test.shape

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train1, y_train1)

In [ ]:
caracteristicas = model.feature_importances_

In [ ]:
argsort = np.argsort(caracteristicas)

In [ ]:
mse = model.train_score_[-1]

In [ ]:
rmse = np.sqrt(mse)
rmse

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
previsoes=(y_pred>0.5)
previsoes

In [ ]:
print(model.score(X_test,y_test))

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

In [ ]:
vai_testar_magica = 300
vai_testar_magica

In [ ]:
#codigo do livro de Aurélion Geron
fig = plt.figure(figsize=(20,5))
plt.bar(list(range(len(y_test[::vai_testar_magica]))), y_test.values[::vai_testar_magica],
        alpha = 0.5, color = 'red', width = 1, label = 'Valores da Base')
plt.bar(list(range(len(y_pred[::vai_testar_magica]))), y_pred[::vai_testar_magica],
        alpha = 0.7, color = 'blue', width = 1, label = 'Valores da Previsão')
plt.legend()